# Call Price Evaluation for BTCC.B

## Get Prices

In [18]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# URL of the page
url = 'https://www.m-x.ca/en/trading/data/quotes?symbol=BTCC*'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('tbody', {'class': 'text-right nowrap'})

# Initialize an empty list to store the data
options_data = []

# Check if the table is found
if table:
    # Iterate over each row in the table
    for row in table.find_all('tr'):
        try:
            # Extract data for each attribute, with error handling
            strike_price = row.find('td', class_='strike_price').text.strip()
            call_bid_price = row.find('td', class_='call bid_price').text.strip()
            call_ask_price = row.find('td', class_='call ask_price').text.strip()
            call_last_price = row.find('td', class_='call last_price').text.strip()

            # Extract expiry date from the data-row attribute
            data_row = json.loads(row['data-row'].replace('&quot;', '"'))
            expiry_date = data_row['call']['expiry_date'] if 'call' in data_row else 'N/A'

            # Append the data to the list
            options_data.append({
                'Date': expiry_date,
                'Strike': strike_price,
                'Bid': call_bid_price,
                'Ask': call_ask_price,
                'Last': call_last_price
            })
        except (AttributeError, KeyError, json.JSONDecodeError) as e:
            print(f"Error extracting data from row: {e}")
else:
    print("Table not found.")

# Convert the list to a DataFrame
df = pd.DataFrame(options_data)

# Convert 'Expiry Date' to datetime and sort by it
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Date', 'Last'], inplace=True)

# Convert 'Strike' and 'Ask' columns to numeric (float)
df['Strike'] = pd.to_numeric(df['Strike'], errors='coerce')
#df['Ask'] = pd.to_numeric(df['Ask'], errors='coerce')
df['Last'] = pd.to_numeric(df['Last'], errors='coerce')
df

,Date,Strike,Bid,Ask,Last
73,2024-03-15,14.0,0,1.00,0.51
72,2024-03-15,13.5,0,0,0.70
71,2024-03-15,13.0,0,0,0.95
70,2024-03-15,12.5,0,0,1.31
69,2024-03-15,12.0,0,0,1.72
...,...,...,...,...,...
302,2026-01-16,9.0,1.80,0,7.40
300,2026-01-16,7.0,4.00,19.95,7.45
299,2026-01-16,6.0,1.08,10.00,8.05
298,2026-01-16,5.0,0,12.00,9.37


## Date of Interest

In [19]:
target_date = '2024-09-20'

filtered_df = df[df['Date'] == pd.to_datetime(target_date)]

# Sort by 'Strike' column from lowest to highest
filtered_df = filtered_df.sort_values(by='Strike')

# Convert filtered DataFrame to list of tuples (Strike, Ask Price)
options = list(filtered_df[['Strike', 'Last']].dropna().apply(tuple, axis=1))

print("Options for", target_date, ":", options)

Options for 2024-09-20 : [(3.0, 10.57), (4.0, 9.57), (4.5, 9.09), (5.0, 8.62), (5.5, 8.21), (6.0, 7.82), (6.5, 7.42), (7.0, 7.04), (7.5, 6.6), (8.0, 6.25), (8.5, 5.9), (9.0, 5.54), (9.5, 5.2), (10.0, 4.9), (11.0, 4.4), (12.0, 3.9), (13.0, 3.48), (14.0, 3.1), (15.0, 2.79), (16.0, 2.4), (17.0, 2.2)]


## Data & Params

In [20]:
# All options expire on January 16th 2026

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz

# Retrieve historical data
ticker = "BTCC-B.TO"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*4)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')

data = yf.download(ticker, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()

daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

risk_free_rate = 0.045

current_stock_price = data['Close'].iloc[-1]  
print(current_stock_price)

[*********************100%%**********************]  1 of 1 completed

13.449999809265137



C:\Users\falty\AppData\Local\Temp\ipykernel_30580\1888896037.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Daily_Return'] = data['Close'].pct_change()


## Profit Based on Final Stock Price

In [22]:
# BTCC-B.TO Call Option Price Evaluation


initial_capital = 4_000
current_stock_price = 13.45
final_stock_price = current_stock_price*1.4
num_shares_per_option = 100

# Function to calculate profit for buying and holding the stock
def calculate_stock_profit(initial_capital, current_price, final_price):
    num_shares = initial_capital // current_price
    return num_shares * (final_price - current_price)

# Function to calculate profit for each option scenario
def calculate_option_profit(initial_capital, strike, ask_price, final_price):
    num_options = initial_capital // (ask_price * num_shares_per_option)
    profit_per_option = max(0, final_price - strike) - ask_price
    return num_options * profit_per_option * num_shares_per_option

# Print final stock price
print(f"Final stock price: ${final_stock_price:.2f}")

# Calculate profit for buying and holding
stock_profit = calculate_stock_profit(initial_capital, current_stock_price, final_stock_price)
print(f"Profit from buying and holding the stock: ${stock_profit:.2f}")

# Calculate and display profit for each option scenario
for strike, ask_price in options:
    option_profit = calculate_option_profit(initial_capital, strike, ask_price, final_stock_price)
    print(f"Profit from option with strike ${strike} and ask price ${ask_price}: ${option_profit:.2f}")


Final stock price: $18.83
Profit from buying and holding the stock: $1597.86
Profit from option with strike $3.0 and ask price $10.57: $1578.00
Profit from option with strike $4.0 and ask price $9.57: $2104.00
Profit from option with strike $4.5 and ask price $9.09: $2096.00
Profit from option with strike $5.0 and ask price $8.62: $2084.00
Profit from option with strike $5.5 and ask price $8.21: $2048.00
Profit from option with strike $6.0 and ask price $7.82: $2505.00
Profit from option with strike $6.5 and ask price $7.42: $2455.00
Profit from option with strike $7.0 and ask price $7.04: $2395.00
Profit from option with strike $7.5 and ask price $6.6: $2838.00
Profit from option with strike $8.0 and ask price $6.25: $2748.00
Profit from option with strike $8.5 and ask price $5.9: $2658.00
Profit from option with strike $9.0 and ask price $5.54: $3003.00
Profit from option with strike $9.5 and ask price $5.2: $2891.00
Profit from option with strike $10.0 and ask price $4.9: $3144.00
P

## Binomial Tree

In [5]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.stats import norm

# Assuming data['Daily_Return'] contains the daily returns
historical_volatility = np.std(np.log(1 + data['Daily_Return'].dropna())) * np.sqrt(252)  # Annualized using trading days in a year
mean_return = np.mean(data['Daily_Return'].dropna()) * 252  # Annualized

historical_skewness = skew(data['Daily_Return'].dropna())
historical_kurtosis = kurtosis(data['Daily_Return'].dropna(), fisher=False)

def binomial_tree_option_price(S, K, T_days, r, sigma, N=20_000, dividends=0, skew=0, kurtosis=0, drift=0):
    T = T_days / 252  # Convert days to years
    dt = T / N
    u = np.exp((drift - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt))  # Incorporating drift
    d = 1 / u
    p = (np.exp((r - dividends) * dt) - d) / (u - d)

    p += skew * 0.001 * (1 - 2 * p) + kurtosis * 0.0005 * (1 - 2 * p)
    p = min(max(p, 0), 1)

    prices = S * d**np.arange(N, -1, -1) * u**np.arange(0, N+1, 1)
    option_values = np.maximum(prices - K, 0)

    discount_factor = np.exp(-r * dt)
    for i in range(N - 1, -1, -1):
        option_values[:i+1] = (p * option_values[1:i+2] + (1 - p) * option_values[:i+1]) * discount_factor
        # For American options, compare with early exercise
        option_values[:i+1] = np.maximum(option_values[:i+1], prices[:i+1] - K)

    return option_values[0]

### Price Specific Date

In [6]:
# Example usage:
risk_free_rate = 0.05  # Example rate
T_days = 6  # days until expiration
N = 100  # number of steps

# Example usage
for strike, ask_price in options:
    option_price = binomial_tree_option_price(
        current_stock_price, strike, T_days, risk_free_rate, historical_volatility, N,
        dividends=0, skew=historical_skewness, kurtosis=historical_kurtosis, drift=mean_return
    )
    print(f"Call Option Price for strike {strike}: ${option_price:.2f}")

Call Option Price for strike 3.0: $10.45
Call Option Price for strike 4.0: $9.46
Call Option Price for strike 4.25: $9.21
Call Option Price for strike 4.5: $8.96
Call Option Price for strike 4.75: $8.71
Call Option Price for strike 5.0: $8.46
Call Option Price for strike 5.25: $8.21
Call Option Price for strike 5.5: $7.96
Call Option Price for strike 5.75: $7.71
Call Option Price for strike 6.0: $7.46
Call Option Price for strike 6.25: $7.21
Call Option Price for strike 6.5: $6.96
Call Option Price for strike 6.75: $6.71
Call Option Price for strike 7.0: $6.46
Call Option Price for strike 7.25: $6.21
Call Option Price for strike 7.5: $5.96
Call Option Price for strike 7.75: $5.71
Call Option Price for strike 8.0: $5.46
Call Option Price for strike 8.25: $5.21
Call Option Price for strike 8.5: $4.96
Call Option Price for strike 8.75: $4.71
Call Option Price for strike 9.0: $4.46
Call Option Price for strike 9.25: $4.21
Call Option Price for strike 9.5: $3.96
Call Option Price for strike

### Look for Undervalued Calls

In [7]:
N = 5_000

for date in df['Date'].unique():
    T_days = (date - pd.Timestamp.now()).days  # Calculate days to expiration
    options_on_date = df[df['Date'] == date]
    options = list(options_on_date[['Strike', 'Ask']].dropna().apply(tuple, axis=1))

    for strike, ask_price in options:
        option_price = binomial_tree_option_price(
            current_stock_price, strike, T_days, risk_free_rate, historical_volatility, N,
            dividends=0, skew=historical_skewness, kurtosis=historical_kurtosis, drift=mean_return
        )

        if ask_price < option_price:
            print(f"Underpriced Call Option for expiration {date.strftime('%Y-%m-%d')}, strike {strike}: Calculated Price: ${option_price:.2f}, Ask Price: ${ask_price}")

Underpriced Call Option for expiration 2024-04-19, strike 9.5: Calculated Price: $4.11, Ask Price: $4.1
Underpriced Call Option for expiration 2024-06-21, strike 5.5: Calculated Price: $8.08, Ask Price: $8.05
Underpriced Call Option for expiration 2024-06-21, strike 6.0: Calculated Price: $7.59, Ask Price: $7.59
Underpriced Call Option for expiration 2024-06-21, strike 7.0: Calculated Price: $6.65, Ask Price: $6.55
Underpriced Call Option for expiration 2024-06-21, strike 9.5: Calculated Price: $4.52, Ask Price: $4.5
Underpriced Call Option for expiration 2024-09-20, strike 3.0: Calculated Price: $10.57, Ask Price: $10.57
Underpriced Call Option for expiration 2024-09-20, strike 4.0: Calculated Price: $9.62, Ask Price: $9.57
Underpriced Call Option for expiration 2024-09-20, strike 4.5: Calculated Price: $9.15, Ask Price: $9.09
Underpriced Call Option for expiration 2024-09-20, strike 5.0: Calculated Price: $8.69, Ask Price: $8.62
Underpriced Call Option for expiration 2024-09-20, stri

## Monte-Carlo Simulation 

In [8]:
%%time
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
from numba import jit, prange

# Calculate log returns
log_returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()

# Identify jumps
jump_threshold = 3 * np.std(log_returns)  # Example threshold
jumps = log_returns[abs(log_returns) > jump_threshold]

# Estimate lambda (jump intensity)
lambda_ = len(jumps) / (len(log_returns) / 365)  # Number of jumps per year

# Estimate mu_J and sigma_J (jump size parameters)
# We use the jumps themselves, not the ratio to their lagged values
mu_J = np.mean(jumps)
sigma_J = np.std(jumps)

@jit(nopython=True, parallel=True)  # Enable JIT compilation with parallel execution
def monte_carlo_option_price(S, K, T_months, r, sigma, lambda_, mu_J, sigma_J, num_paths=10000, num_steps=100, skew=0, kurtosis=0):
    """
    Monte Carlo simulation for European call option pricing with jump diffusion.

    Parameters:
    S (float): Current stock price.
    K (float): Strike price of the option.
    T_months (int): Time to maturity in months.
    r (float): Risk-free interest rate.
    sigma (float): Volatility of the underlying asset.
    lambda_ (float): Jump intensity, representing the frequency of jumps.
    mu_J (float): Mean of the logarithm of the jump size.
    sigma_J (float): Standard deviation of the logarithm of the jump size.
    num_paths (int): Number of simulated paths in the Monte Carlo simulation.
    num_steps (int): Number of time steps in each simulated path.
    skew (float): Skewness of the asset returns. Defaults to 0 for no skew.
    kurtosis (float): Kurtosis of the asset returns. Defaults to 0 for normal kurtosis.

    Returns:
    float: Estimated price of the European call option.
    """
    T = T_months / 365  # Convert maturity from months to years
    dt = T / num_steps  # Time step for simulation
    discount_factor = np.exp(-r * T)  # Discount factor for present value

    payoffs = np.zeros(num_paths)  # Initialize array for option payoffs
    for i in prange(num_paths):  # Parallel loop for each path
        S_t = S  # Initial stock price for this path
        for j in range(num_steps):  # Time step loop
            z = np.random.normal()  # Standard normal random variable
            shock = z + skew * (z ** 2 - 1)  # Adjust random variable for skew and kurtosis
            num_jumps = np.random.poisson(lambda_ * dt)  # Poisson process for number of jumps
            jump_sum = np.sum(np.random.lognormal(mu_J, sigma_J, num_jumps) - 1)  # Sum of jump sizes
            S_t *= np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * shock + jump_sum)  # Stock price update
        payoffs[i] = max(S_t - K, 0)  # Call option payoff for this path

    return discount_factor * np.mean(payoffs)  # Average discounted payoff



CPU times: total: 125 ms
Wall time: 145 ms


### Price Specific Date

In [9]:
# Example usage
for strike, ask_price in options:
    T_days = 6 
    option_price = monte_carlo_option_price(
        current_stock_price, strike, T_months, risk_free_rate, historical_volatility,
        lambda_, mu_J, sigma_J,
        num_paths=1_000_000, num_steps=T_days*24,
        skew=historical_skewness, kurtosis=historical_kurtosis
    )
    print(f"Monte Carlo Call Option Price for strike {strike}: ${option_price:.2f}")

NameError: name 'T_months' is not defined

### Look for Undervalued Calls

In [ ]:
for date in df['Date'].unique():
    T_days = (date - pd.Timestamp.now()).days  # Calculate days to expiration
    options_on_date = df[df['Date'] == date]
    options = list(options_on_date[['Strike', 'Ask']].dropna().apply(tuple, axis=1))

    for strike, ask_price in options:
        option_price = monte_carlo_option_price(
            current_stock_price, strike, T_days, risk_free_rate, historical_volatility,
            lambda_, mu_J, sigma_J,
            num_paths=100_000, num_steps=T_days * 24,  # Assuming 24 hours in a day for finer granularity
            skew=historical_skewness, kurtosis=historical_kurtosis
        )

        if ask_price < option_price:
            print(f"Underpriced Call Option for expiration {date.strftime('%Y-%m-%d')}, strike {strike}: Calculated Price: ${option_price:.2f}, Ask Price: ${ask_price}")